In [1]:
def searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
    

In [2]:
def select_first(driver):
    
    driver.implicitly_wait(10)
    
    first = driver.find_element_by_class_name('eLAPa')
    print("첫게시물클릭햇다")
    first.click()
    time.sleep(3)

In [3]:
import re

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)  
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    data = [content, date, like, place, tags]
    return data


In [4]:
def move_next(driver):

    driver.implicitly_wait(10)
    
    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

driver = webdriver.Chrome('./webdriver/chromedriver_win32/chromedriver.exe') 
usr = '01027047772'
pwd = open('./pass.txt').readline()
word = "김선호"    
url = searching(word)
driver.implicitly_wait(3)
driver.get(url)
time.sleep(3)

element_id = driver.find_element_by_name("username")
element_id.send_keys(usr)
element_password = driver.find_element_by_name("password")
element_password.send_keys(pwd)
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

time.sleep(5)

driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm').click()
print(url,"url")
print("첫게시물 등장")

driver.get(url)
select_first(driver)

results = [ ]

target = 50
for i in range(target):
    data = get_content(driver)
    results.append(data)
    move_next(driver)

print(results[:2])

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="username"]"}
  (Session info: chrome=89.0.4389.90)


In [52]:

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./files/오늘뭐먹지.xlsx')

In [42]:
pohang_insta_df = pd.DataFrame( [ ] )

folder = './files/'

f_list = ['kimseonho.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    pohang_insta_df = pohang_insta_df.append(temp)

pohang_insta_df.columns =['','content','data','like','place','tags']


In [43]:
pohang_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
pohang_insta_df.to_excel('./files/test_crawling_raw.xlsx', index = False)